#### Recurrent Neural Network

by Piyush Sanghi

###### 1. Data Processing: 
This data set is a bit messy, so the preprocessing portion is largely a tutorial to make sure students have data ready for keras. 

###### a) Import the following libraries:

In [1]:
import sys
import os
import json
import pandas
import numpy
import optparse

from keras.callbacks import TensorBoard
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, SimpleRNN
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from collections import OrderedDict

Using TensorFlow backend.


###### b) We will read the code in slightly differently than before: 

In [2]:
dataframe = pandas.read_csv("dev-access.csv", engine='python', quotechar='|', header=None)

###### c) We then need to convert to a numpy.ndarray type: 

In [3]:
dataset = dataframe.values

In [4]:
dataframe.head()

,0,1
0,"{""timestamp"":1502738402847,""method"":""post"",""qu...",0
1,"{""timestamp"":1502738402849,""method"":""post"",""qu...",0
2,"{""timestamp"":1502738402852,""method"":""post"",""qu...",0
3,"{""timestamp"":1502738402852,""method"":""post"",""qu...",0
4,"{""timestamp"":1502738402853,""method"":""post"",""qu...",0


###### d) Check the shape of the data set - it should be (26773, 2). Spend some time looking at the data. 

In [5]:
dataset.shape

(26773, 2)

###### e) Store all rows and the 0th index as the feature data: 

In [6]:
X = dataset[:,0]

###### f) Store all rows and index 1 as the target variable: 

In [7]:
Y = dataset[:,1]

###### g) In the next step, we will clean up the predictors. This includes removing features that are not valuable, such as timestamp and source. 

In [8]:
for index, item in enumerate(X):
    # Quick hack to space out json elements
    reqJson = json.loads(item, object_pairs_hook=OrderedDict)
    del reqJson['timestamp']
    del reqJson['headers']
    del reqJson['source']
    del reqJson['route']
    del reqJson['responsePayload']
    X[index] = json.dumps(reqJson, separators=(',', ':'))

###### h) We next will tokenize our data, which just means vectorizing our text. Given the data we will tokenize every character (thus char_level = True)

In [9]:
tokenizer = Tokenizer(filters='\t\n', char_level=True)
tokenizer.fit_on_texts(X)

# we will need this later
num_words = len(tokenizer.word_index)+1
X = tokenizer.texts_to_sequences(X)

###### i) Need to pad our data as each observation has a different length

In [10]:
max_log_length = 1024
X_processed = sequence.pad_sequences(X, maxlen=max_log_length)

###### j) Create your train set to be 75% of the data and your test set to be 25%

In [11]:
numpy.random.shuffle(X_processed)
X_train, X_test = X_processed[:int(0.75*X_processed.shape[0]),], X_processed[int(0.75*X_processed.shape[0]):,]
y_train, y_test = Y[:int(0.75*Y.shape[0]),], Y[int(0.75*Y.shape[0]):,]

In [12]:
X_processed.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape


((26773, 1024), (20079, 1024), (6694, 1024), (20079,), (6694,))

#### 2. Model 1 - RNN: 
The first model will be a pretty minimal RNN with only an embedding layer, simple RNN and Dense layer. The next model we will add a few more layers. 

###### a) Start by creating an instance of a Sequential model

In [13]:
model = Sequential()

###### b) From there, add an Embedding layer: https://keras.io/layers/embeddings/ (Links to an external site.)Links to an external site.

Params:
- input_dim = num_words (the variable we created above)
- output_dim = 32
- input_length = max_log_length (we also created this above) 
- Keep all other variables as the defaults (shown below)

In [14]:
model.add(Embedding(input_dim=num_words, output_dim=32, input_length=max_log_length))

###### c) Add a SimpleRNN layer: https://keras.io/layers/recurrent/ (Links to an external site.)Links to an external site.

Params:
- units = 32
- activation = 'relu'

In [15]:
model.add(SimpleRNN(units=32, activation='relu'))

###### d) Finally, we will add a Dense layer: https://keras.io/layers/core/#dense (Links to an external site.)Links to an external site.

Params:
- units = 1 (this will be our output)
- activation = sigmoid

In [16]:
model.add(Dense(units=1, activation='sigmoid'))

###### e) Compile model using the .compile() method: https://keras.io/models/model/ (Links to an external site.)Links to an external site.

Params:
- loss = binary_crossentropy
- optimizer = adam
- metrics = accuracy

In [17]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

###### f) Print the model summary

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1024, 32)          2016      
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 4,129
Trainable params: 4,129
Non-trainable params: 0
_________________________________________________________________


###### g) Use the .fit() method to fit the model on the train data. Use a validation split of 0.25, epochs=3 and batch size = 128.

In [19]:
model.fit(X_train, y_train, epochs=3, batch_size=128, validation_split=0.25)

Train on 15059 samples, validate on 5020 samples
Epoch 1/3
15059/15059 [==============================] - 117s 8ms/step - loss: 0.6799 - acc: 0.5822 - val_loss: 0.7587 - val_acc: 0.3400
Epoch 2/3
15059/15059 [==============================] - 116s 8ms/step - loss: 0.6797 - acc: 0.5822 - val_loss: 0.7557 - val_acc: 0.3400
Epoch 3/3
15059/15059 [==============================] - 115s 8ms/step - loss: 0.6797 - acc: 0.5822 - val_loss: 0.7488 - val_acc: 0.3400


###### h) Use the .evaluate() method to get the loss value & the accuracy value on the test data. Use a batch size of 128 again.

In [20]:
model.evaluate(X_test, y_test, batch_size=128)

6694/6694 [==============================] - 25s 4ms/step


[0.7204432913998472, 0.4390498954287422]

In [21]:
model.metrics_names

['loss', 'acc']

#### 3) Model 2 - RNN + Dropout Layers + New Activation Function:

Now we will add a few new layers to our RNN and incorporate the more powerful LSTM. You will be creating a new model here, so make sure to call it something different than the model from Part 2.

###### a) This RNN needs to have the following layers (add in this order):

- Embedding Layer (use same params as before)
- LSTM Layer (units = 64, recurrent_dropout = 0.5)
- Dropout Layer - use a value of 0.5 
- Dense Layer - (use same params as before)

In [22]:
model2 = Sequential()
model2.add(Embedding(input_dim=num_words, output_dim=32, input_length=max_log_length))
model2.add(LSTM(units=64, recurrent_dropout=0.5))
model2.add(Dropout(0.5))
model2.add(Dense(units=1, activation='sigmoid'))

###### b) Compile model using the .compile() method:
    Params:
- loss = binary_crossentropy
- optimizer = adam
- metrics = accuracy

In [23]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

###### c) Print the model summary

In [24]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1024, 32)          2016      
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 26,913
Trainable params: 26,913
Non-trainable params: 0
_________________________________________________________________


###### d) Use the .fit() method to fit the model on the train data. Use a validation split of 0.25, epochs=3 and batch size = 128.

In [25]:
model.fit(X_train, y_train, batch_size=128, validation_split=0.25)

Train on 15059 samples, validate on 5020 samples
Epoch 1/1
15059/15059 [==============================] - 115s 8ms/step - loss: 0.6798 - acc: 0.5822 - val_loss: 0.7568 - val_acc: 0.3400


###### e) Use the .evaluate() method to get the loss value & the accuracy value on the test data. Use a batch size of 128 again.

In [26]:
model2.evaluate(X_test, y_test, batch_size=128)

6694/6694 [==============================] - 49s 7ms/step


[0.6934225755304873, 0.44801314610209897]

In [27]:
model2.metrics_names

['loss', 'acc']

#### 4) Recurrent Neural Net Model 3: Build Your Own

###### a) RNN Requirements: 
- Use 5 or more layers
- Add a layer that was not utilized in Model 1 or Model 2 (Note: This could be a new Dense layer or an additional LSTM)

In [28]:
model3 = Sequential()
model3.add(Embedding(input_dim=num_words, output_dim=32, input_length=max_log_length))
model3.add(LSTM(units=64, recurrent_dropout=.05))
model3.add(Dense(units=64, activation='relu'))
model3.add(Dropout(0.4))
model3.add(Dense(units=64, activation='relu'))
model3.add(Dropout(0.3))
model3.add(Dense(units=64, activation='relu'))
model3.add(Dropout(0.3))
model3.add(Dense(units=64, activation='relu'))
model3.add(Dense(units=1, activation='relu'))

###### b) Compiler Requirements: 
- Try a new optimizer for the compile step
- Keep accuracy as a metric (feel free to add more metrics if desired)

In [29]:
model3.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

###### c) Print the model summary

In [30]:
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1024, 32)          2016      
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
__________

###### d) Use the .fit() method to fit the model on the train data. Use a validation split of 0.25, epochs=3 and batch size = 128.

In [31]:
model3.fit(X_train, y_train, epochs=3, batch_size=128, validation_split=0.25)

Train on 15059 samples, validate on 5020 samples
Epoch 1/3
15059/15059 [==============================] - 293s 19ms/step - loss: 0.7560 - acc: 0.5569 - val_loss: 0.7638 - val_acc: 0.3400
Epoch 2/3
15059/15059 [==============================] - 287s 19ms/step - loss: 0.6865 - acc: 0.5711 - val_loss: 0.7700 - val_acc: 0.3400
Epoch 3/3
15059/15059 [==============================] - 286s 19ms/step - loss: 0.6828 - acc: 0.5797 - val_loss: 0.7858 - val_acc: 0.3400


###### e) Use the .evaluate() method to get the loss value & the accuracy value on the test data. Use a batch size of 128 again.

In [32]:
model3.evaluate(X_test, y_test, batch_size=128)

6694/6694 [==============================] - 49s 7ms/step


[0.7428522710661978, 0.4390498954287422]

In [33]:
model3.metrics_names

['loss', 'acc']